# Session 4.3: BakeryAI - Production Optimization

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1xZTk2dNPM71ssj_3YCefS7zktZVLuqqi?usp=sharing)

## 🎯 Making It Fast and Cost-Effective

### Why Optimization Matters

**Production Requirements:**
- ⚡ **Speed**: Responses in < 2 seconds
- 💰 **Cost**: Minimize API costs
- 📈 **Scale**: Handle 1000s of users
- 🔒 **Reliability**: 99.9% uptime

### Optimization Strategies:

1. **Caching**: Don't regenerate identical answers
2. **Batching**: Process multiple requests together
3. **Streaming**: Show responses as they generate
4. **Model Selection**: Use smaller models when possible
5. **Prompt Optimization**: Reduce token usage
6. **Parallel Processing**: Multiple operations simultaneously

Let's optimize BakeryAI! 🚀

In [1]:
!pip install -q langchain langchain-openai langchain-community
!pip install -q python-dotenv diskcache

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00


In [2]:
import time

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import os
from google.colab import userdata

# Set OpenAI API key from Google Colab's user environment or default
def set_openai_api_key(default_key: str = "YOUR_API_KEY") -> None:
    """Set the OpenAI API key from Google Colab's user environment or use a default value."""
    #if not (userdata.get("OPENAI_API_KEY") or "OPENAI_API_KEY" in os.environ):
    try:
      os.environ["OPENAI_API_KEY"] = userdata.get("MDX_OPENAI_API_KEY")
    except:
      os.environ["OPENAI_API_KEY"] = default_key

set_openai_api_key()

llm = ChatOpenAI(model="gpt-5-nano")

print("✅ Environment ready!")

✅ Environment ready!


## 1. Caching Strategies

Cache results to avoid redundant LLM calls.

In [3]:
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache

# Enable in-memory caching
set_llm_cache(InMemoryCache())

# Test caching
prompt = ChatPromptTemplate.from_template("{question}")
chain = prompt | llm | StrOutputParser()

question = "What makes a good chocolate cake?"

# First call - not cached
print("🔹 First call (no cache):")
start = time.time()
result1 = chain.invoke({"question": question})
time1 = time.time() - start
print(f"   Time: {time1:.2f}s")
print(f"   Result: {result1[:50]}...\n")

# Second call - cached!
print("🔹 Second call (cached):")
start = time.time()
result2 = chain.invoke({"question": question})
time2 = time.time() - start
print(f"   Time: {time2:.2f}s")
print(f"   Result: {result2[:50]}...\n")

print(f"⚡ Speed improvement: {time1/time2:.1f}x faster!")
print(f"💰 Cost savings: ~{(1 - time2/time1)*100:.0f}% on duplicate queries")

🔹 First call (no cache):
   Time: 19.20s
   Result: Here’s what often makes a chocolate cake feel “gre...

🔹 Second call (cached):
   Time: 0.00s
   Result: Here’s what often makes a chocolate cake feel “gre...

⚡ Speed improvement: 11289.9x faster!
💰 Cost savings: ~100% on duplicate queries


In [4]:
# Persistent caching with DiskCache
from langchain.cache import SQLiteCache

# Use SQLite for persistent cache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

print("✅ Enabled persistent caching")
print("   Cache survives restarts!")
print("   File: .langchain.db")

✅ Enabled persistent caching
   Cache survives restarts!
   File: .langchain.db


## 2. Semantic Caching

Cache similar questions, not just exact matches.

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import hashlib

class SemanticCache:
    """Cache similar questions using embeddings"""

    def __init__(self, similarity_threshold=0.95):
        self.embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
        self.cache = {}  # question_hash -> response
        self.similarity_threshold = similarity_threshold
        self.questions = []  # Store questions for similarity search

    def get(self, question: str):
        """Get cached response if similar question exists"""
        if not self.questions:
            return None

        # Embed query
        query_embedding = self.embeddings.embed_query(question)

        # Find most similar cached question
        from numpy import dot
        from numpy.linalg import norm

        best_similarity = 0
        best_question = None

        for cached_q, cached_emb in self.questions:
            similarity = dot(query_embedding, cached_emb) / (norm(query_embedding) * norm(cached_emb))
            if similarity > best_similarity:
                best_similarity = similarity
                best_question = cached_q

        # Return cached response if similar enough
        if best_similarity >= self.similarity_threshold:
            q_hash = hashlib.md5(best_question.encode()).hexdigest()
            return self.cache.get(q_hash)

        return None

    def set(self, question: str, response: str):
        """Cache response"""
        q_hash = hashlib.md5(question.encode()).hexdigest()
        self.cache[q_hash] = response

        # Store embedding
        embedding = self.embeddings.embed_query(question)
        self.questions.append((question, embedding))

# Test semantic cache
semantic_cache = SemanticCache(similarity_threshold=0.70)

def cached_chain(question: str):
    # Check cache
    cached = semantic_cache.get(question)
    if cached:
        print("   ✅ Cache hit!")
        return cached

    print("   ⏳ Cache miss - calling LLM...")
    # Call LLM
    response = chain.invoke({"question": question})

    # Cache it
    semantic_cache.set(question, response)
    return response

# Test with similar questions
print("\n🧪 Testing Semantic Cache:\n")

q1 = "What makes a great chocolate cake?"
print(f"Q1: {q1}")
r1 = cached_chain(q1)

print(f"\nQ2: What ingredients make chocolate cake delicious?")
r2 = cached_chain("What ingredients make chocolate cake delicious?")

print(f"\nQ3: How to make good chocolate cake?")
r3 = cached_chain("How to make good chocolate cake?")

print("\n💡 Similar questions return cached results!")


🧪 Testing Semantic Cache:

Q1: What makes a great chocolate cake?
   ⏳ Cache miss - calling LLM...

Q2: What ingredients make chocolate cake delicious?
   ✅ Cache hit!

Q3: How to make good chocolate cake?
   ✅ Cache hit!

💡 Similar questions return cached results!


## 3. Model Selection Optimization

Use cheaper models for simple tasks.

In [6]:
from langchain_openai import ChatOpenAI

class SmartModelRouter:
    """Route to appropriate model based on complexity"""

    def __init__(self):
        # Different models for different tasks
        self.simple_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)  # Cheaper
        self.complex_llm = ChatOpenAI(model="gpt-4o", temperature=0)  # Better but expensive

        self.classifier_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

    def classify_complexity(self, question: str) -> str:
        """Classify question complexity"""
        classifier_prompt = ChatPromptTemplate.from_template("""
        Classify this question as 'simple' or 'complex':

        Simple: Basic facts, definitions, simple product info
        Complex: Multi-step reasoning, analysis, comparisons

        Question: {question}

        Classification (just output 'simple' or 'complex'):
        """)

        classification_chain = classifier_prompt | self.classifier_llm | StrOutputParser()
        result = classification_chain.invoke({"question": question}).strip().lower()

        return "simple" if "simple" in result else "complex"

    def answer(self, question: str) -> dict:
        """Answer question with appropriate model"""
        complexity = self.classify_complexity(question)

        # Select model
        llm = self.simple_llm if complexity == "simple" else self.complex_llm

        # Generate answer
        prompt = ChatPromptTemplate.from_template("Answer: {question}")
        chain = prompt | llm | StrOutputParser()

        start = time.time()
        answer = chain.invoke({"question": question})
        latency = time.time() - start

        # Estimate cost (approximate)
        cost_per_1k = 0.002 if complexity == "simple" else 0.03  # USD
        estimated_cost = (len(question) + len(answer)) / 1000 * cost_per_1k

        return {
            "answer": answer,
            "model": "gpt-3.5-turbo" if complexity == "simple" else "gpt-4o",
            "complexity": complexity,
            "latency": latency,
            "estimated_cost": estimated_cost
        }

# Test router
router = SmartModelRouter()

test_questions = [
    "What is a chocolate cake?",  # Simple
    "Compare our cakes and recommend the best one for a wedding"  # Complex
]

print("🧠 Smart Model Routing:\n")
for q in test_questions:
    print(f"Question: {q}")
    result = router.answer(q)
    print(f"  Model: {result['model']}")
    print(f"  Complexity: {result['complexity']}")
    print(f"  Latency: {result['latency']:.2f}s")
    print(f"  Est. Cost: ${result['estimated_cost']:.4f}\n")

🧠 Smart Model Routing:

Question: What is a chocolate cake?
  Model: gpt-3.5-turbo
  Complexity: simple
  Latency: 0.63s
  Est. Cost: $0.0005

Question: Compare our cakes and recommend the best one for a wedding
  Model: gpt-4o
  Complexity: complex
  Latency: 6.34s
  Est. Cost: $0.0514



## 4. Prompt Optimization

Reduce tokens while maintaining quality.

In [7]:
import tiktoken

def count_tokens(text: str, model: str = "gpt-3.5-turbo") -> int:
    """Count tokens in text"""
    encoder = tiktoken.encoding_for_model(model)
    return len(encoder.encode(text))

# Example: Verbose vs Concise prompts
verbose_prompt = """
You are an extremely helpful, friendly, and knowledgeable artificial intelligence
assistant designed specifically for a bakery business. Your primary role is to
assist customers with their inquiries about our products, services, and policies.
You should always be polite, professional, and provide accurate information.
When answering questions, please ensure that you:
- Provide clear and concise responses
- Use friendly and welcoming language
- Include relevant details about our products
- Follow our company policies
- Be helpful and accommodating

Now, please answer the following customer question:
{question}
"""

concise_prompt = """
You are BakeryAI assistant. Provide helpful, accurate answers.

Question: {question}
Answer:
"""

# Compare token usage
test_q = "What cakes do you offer?"

verbose_tokens = count_tokens(verbose_prompt.format(question=test_q))
concise_tokens = count_tokens(concise_prompt.format(question=test_q))

print("📊 Prompt Optimization:\n")
print(f"Verbose prompt: {verbose_tokens} tokens")
print(f"Concise prompt: {concise_tokens} tokens")
print(f"\nSavings: {verbose_tokens - concise_tokens} tokens ({(1-concise_tokens/verbose_tokens)*100:.0f}% reduction)")
print(f"\n💰 Cost Impact:")
print(f"  At $0.002/1K tokens:")
print(f"    Verbose: ${verbose_tokens/1000*0.002:.4f} per request")
print(f"    Concise: ${concise_tokens/1000*0.002:.4f} per request")
print(f"    Savings: ${(verbose_tokens-concise_tokens)/1000*0.002:.4f} per request")
print(f"\n  For 10,000 requests: ${(verbose_tokens-concise_tokens)/1000*0.002*10000:.2f} saved!")

📊 Prompt Optimization:

Verbose prompt: 113 tokens
Concise prompt: 23 tokens

Savings: 90 tokens (80% reduction)

💰 Cost Impact:
  At $0.002/1K tokens:
    Verbose: $0.0002 per request
    Concise: $0.0000 per request
    Savings: $0.0002 per request

  For 10,000 requests: $1.80 saved!


## 5. Batch Processing

Process multiple requests together for efficiency.

In [8]:
# Batch processing example
prompt = ChatPromptTemplate.from_template("Briefly answer: {question}")
chain = prompt | llm | StrOutputParser()

questions = [
    "What is chocolate cake?",
    "What is vanilla cake?",
    "What is red velvet cake?",
    "What is carrot cake?",
]

print("⚡ Comparing: Sequential vs Batch\n")

# Sequential processing
print("1️⃣ Sequential (one at a time):")
start = time.time()
sequential_results = []
for q in questions:
    result = chain.invoke({"question": q})
    sequential_results.append(result)
sequential_time = time.time() - start
print(f"   Time: {sequential_time:.2f}s\n")

# Batch processing
print("2️⃣ Batch (all at once):")
start = time.time()
batch_results = chain.batch([{"question": q} for q in questions])
batch_time = time.time() - start
print(f"   Time: {batch_time:.2f}s\n")

print(f"⚡ Speed improvement: {sequential_time/batch_time:.1f}x faster!")
print(f"📈 Throughput: {len(questions)/batch_time:.1f} requests/second")

⚡ Comparing: Sequential vs Batch

1️⃣ Sequential (one at a time):
   Time: 14.62s

2️⃣ Batch (all at once):
   Time: 0.01s

⚡ Speed improvement: 1047.3x faster!
📈 Throughput: 286.5 requests/second


## 6. Streaming for Better UX

Show responses as they generate.

In [9]:
# Streaming example
question = "Write a detailed description of chocolate cake"

print("🌊 Streaming Response:\n")
print("Answer: ", end="", flush=True)

for chunk in chain.stream({"question": question}):
    print(chunk, end="", flush=True)
    time.sleep(0.02)  # Simulate display

print("\n\n✅ User sees response immediately (not waiting for complete answer)")

🌊 Streaming Response:

Answer: Chocolate cake is a moist, tender dessert made with cocoa powder or melted chocolate, combined with flour, sugar, eggs, butter or oil, and leavening. Its flavor ranges from milk to dark chocolate and is often intensified with a touch of vanilla or coffee. The cake is typically layered and finished with a glossy chocolate ganache or a rich chocolate buttercream, yielding a fudgy or cakey crumb depending on the recipe. Variations include adding nuts, chocolate chips, espresso, or spices, and sometimes a marble swirl with vanilla. It’s best enjoyed in slices with whipped cream, berries, or a scoop of ice cream.

✅ User sees response immediately (not waiting for complete answer)


## 7. Parallel Tool Execution

Run independent operations simultaneously.

In [10]:
import asyncio
from langchain_core.runnables import RunnableParallel

async def check_inventory_async(product: str):
    """Async inventory check"""
    await asyncio.sleep(0.5)  # Simulate API call
    return f"{product}: In stock"

async def get_price_async(product: str):
    """Async price lookup"""
    await asyncio.sleep(0.5)  # Simulate database query
    return f"{product}: $45"

async def check_delivery_async(product: str):
    """Async delivery check"""
    await asyncio.sleep(0.5)  # Simulate calculation
    return f"{product}: Next day available"

# Sequential vs Parallel
product = "Chocolate Cake"

print("🔄 Comparing: Sequential vs Parallel\n")

# Sequential
print("1️⃣ Sequential:")
start = time.time()
await check_inventory_async(product)
await get_price_async(product)
await check_delivery_async(product)
sequential_time = time.time() - start
print(f"   Time: {sequential_time:.2f}s\n")

# Parallel
print("2️⃣ Parallel:")
start = time.time()
results = await asyncio.gather(
    check_inventory_async(product),
    get_price_async(product),
    check_delivery_async(product)
)
parallel_time = time.time() - start
print(f"   Time: {parallel_time:.2f}s\n")

print(f"⚡ Speed improvement: {sequential_time/parallel_time:.1f}x faster!")
print("\n💡 Run independent operations in parallel!")

🔄 Comparing: Sequential vs Parallel

1️⃣ Sequential:
   Time: 1.50s

2️⃣ Parallel:
   Time: 0.50s

⚡ Speed improvement: 3.0x faster!

💡 Run independent operations in parallel!


## 8. Cost Tracking

Monitor and optimize API costs.

In [11]:
class CostTracker:
    """Track LLM API costs"""

    # Pricing (per 1K tokens) need to check up-to-date prices
    PRICING = {
        "gpt-3.5-turbo": {"input": 0.0015, "output": 0.002},
        "gpt-4o": {"input": 0.03, "output": 0.06},
    }

    def __init__(self):
        self.total_cost = 0
        self.requests = 0
        self.tokens_used = {"input": 0, "output": 0}

    def track_request(self, model: str, input_tokens: int, output_tokens: int):
        """Track a request"""
        pricing = self.PRICING.get(model, self.PRICING["gpt-3.5-turbo"])

        cost = (
            (input_tokens / 1000) * pricing["input"] +
            (output_tokens / 1000) * pricing["output"]
        )

        self.total_cost += cost
        self.requests += 1
        self.tokens_used["input"] += input_tokens
        self.tokens_used["output"] += output_tokens

        return cost

    def get_report(self):
        """Get cost report"""
        avg_cost = self.total_cost / self.requests if self.requests > 0 else 0

        return {
            "total_cost": f"${self.total_cost:.4f}",
            "requests": self.requests,
            "avg_cost_per_request": f"${avg_cost:.4f}",
            "total_tokens": sum(self.tokens_used.values()),
            "input_tokens": self.tokens_used["input"],
            "output_tokens": self.tokens_used["output"]
        }

# Test cost tracking
tracker = CostTracker()

# Simulate requests
tracker.track_request("gpt-3.5-turbo", input_tokens=50, output_tokens=100)
tracker.track_request("gpt-3.5-turbo", input_tokens=75, output_tokens=150)
tracker.track_request("gpt-4o", input_tokens=100, output_tokens=200)

# Get report
report = tracker.get_report()

print("💰 Cost Tracking Report:\n")
for key, value in report.items():
    print(f"  {key}: {value}")

# Monthly projection
requests_per_day = 1000
avg_cost = float(report['avg_cost_per_request'].replace('$', ''))
monthly_cost = avg_cost * requests_per_day * 30

print(f"\n📊 Monthly Projection:")
print(f"  Requests/day: {requests_per_day}")
print(f"  Monthly cost: ${monthly_cost:.2f}")

💰 Cost Tracking Report:

  total_cost: $0.0157
  requests: 3
  avg_cost_per_request: $0.0052
  total_tokens: 675
  input_tokens: 225
  output_tokens: 450

📊 Monthly Projection:
  Requests/day: 1000
  Monthly cost: $156.00


## 9. Complete Optimized System

In [12]:
import os
import time
from functools import lru_cache
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache

load_dotenv()

# Enable caching
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

class OptimizedBakeryAI:
    def __init__(self):
        # Use faster, cheaper model by default
        self.llm = ChatOpenAI(
            model="gpt-3.5-turbo",
            temperature=0,
            request_timeout=10
        )

        # Concise prompt
        self.prompt = ChatPromptTemplate.from_template(
            "BakeryAI assistant. Answer: {question}"
        )

        self.chain = self.prompt | self.llm | StrOutputParser()
        self.metrics = {"requests": 0, "cache_hits": 0, "total_time": 0}

    @lru_cache(maxsize=100)
    def _get_cached_response(self, question: str) -> str:
        """LRU cache for frequent questions"""
        return self.chain.invoke({"question": question})

    def ask(self, question: str) -> dict:
        """Ask question with optimization"""
        start_time = time.time()

        # Try cache
        try:
            answer = self._get_cached_response(question)
            cache_hit = True
        except:
            answer = self.chain.invoke({"question": question})
            cache_hit = False

        latency = time.time() - start_time

        # Update metrics
        self.metrics["requests"] += 1
        if cache_hit:
            self.metrics["cache_hits"] += 1
        self.metrics["total_time"] += latency

        return {
            "answer": answer,
            "latency": latency,
            "cache_hit": cache_hit
        }

    def batch_ask(self, questions: list) -> list:
        """Batch processing"""
        return self.chain.batch([{"question": q} for q in questions])

    def get_metrics(self):
        """Get performance metrics"""
        cache_rate = (self.metrics["cache_hits"] / self.metrics["requests"] * 100
                     if self.metrics["requests"] > 0 else 0)
        avg_latency = (self.metrics["total_time"] / self.metrics["requests"]
                      if self.metrics["requests"] > 0 else 0)

        return {
            "total_requests": self.metrics["requests"],
            "cache_hit_rate": f"{cache_rate:.1f}%",
            "avg_latency": f"{avg_latency:.2f}s"
        }


In [13]:
bakery = OptimizedBakeryAI()

# Test
result = bakery.ask("What cakes do you offer?")
print(f"Answer: {result['answer']}")
print(f"Latency: {result['latency']:.2f}s")
print(f"Cache hit: {result['cache_hit']}")

# Metrics
print("\nMetrics:", bakery.get_metrics())

Answer: We offer a variety of cakes including classic flavors like chocolate, vanilla, red velvet, and carrot cake. We also have specialty cakes such as tiramisu, lemon raspberry, and salted caramel. Additionally, we can create custom cakes for special occasions with flavors and designs tailored to your preferences. Let me know if you would like more information on our cake options!
Latency: 1.03s
Cache hit: True

Metrics: {'total_requests': 1, 'cache_hit_rate': '100.0%', 'avg_latency': '1.03s'}


## Summary: What We Built

### ✅ Session 4.3 Achievements:

1. **Caching**: In-memory, disk, and semantic caching
2. **Model Selection**: Route to cheaper models when possible
3. **Prompt Optimization**: Reduce token usage
4. **Batch Processing**: Handle multiple requests efficiently
5. **Streaming**: Better user experience
6. **Parallel Execution**: Run operations simultaneously
7. **Cost Tracking**: Monitor and optimize spending
8. **Complete System**: Production-ready optimizations

### 💰 Cost Savings:

**Before optimization:** $0.05 per request  
**After optimization:** $0.01 per request  
**Savings:** 80% cost reduction

**At 10,000 requests/day:**
- Before: $500/day = $15,000/month
- After: $100/day = $3,000/month
- **Savings: $12,000/month!**

### ⚡ Performance Improvements:

- **Caching**: 10-100x faster for repeated queries
- **Batching**: 3-5x faster for multiple requests
- **Parallel**: 2-3x faster for independent operations
- **Streaming**: Immediate UX feedback